In [5]:
import pandas as pd
import os 
import requests
from dotenv import load_dotenv
load_dotenv()


def get_data(text, API_KEY):
  # Define the request parameters
  payload = {
      "textQuery": text
  }

  headers = {
      "Content-Type": "application/json",
      "X-Goog-Api-Key": API_KEY,
      "X-Goog-FieldMask": "places.name,places.id,places.types,places.formattedAddress,places.location,places.rating,places.userRatingCount,places.displayName,places.reviews"
  }

  # Send the POST request
  response = requests.post(
      "https://places.googleapis.com/v1/places:searchText", json=payload, headers=headers
  )

  # Check for successful response
  if response.status_code == 200:
    # Get the response data
    data = response.json()
  else:
    print(f"Error: {response.status_code}")
    raise ConnectionError(response.text)
  
  return data 



In [ ]:
paths = ['raw', 'data']

for path in paths:
    os.makedirs(f"{path}", exist_ok=True)

In [6]:
# Replace 'API_KEY' with your actual Google Places API Key
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

query = "Restaurants in Abu Dhabi"

data = get_data(query, API_KEY=API_KEY)

df = pd.DataFrame(data['places'])
df.to_parquet(f"./raw/{query}", index=False)

df['latitude'] = df.location.apply(lambda x: x['latitude'])
df['longitude'] = df.location.apply(lambda x: x['longitude'])
df  = df.drop('location', axis=1)
def process_reviews(reviews):
    return ' '.join((f"Rating: {review['rating']}\n{review['text']['text']}\n"
                        if "text" in review
                        else "") for review in reviews if review)

df['reviews'] = df['reviews'].apply(process_reviews)
df['query'] = query
df.to_parquet(f"./data/{query}", index=False)

In [12]:
import pandas as pd


df = pd.read_parquet(f"./raw/Restaurants in Abu Dhabi")
df['reviews'] = df.reviews.apply(lambda x: [f"Rating: {review['rating']}\n{review['text']['text']}\n"
                        if "text" for review in x])
df.reviews


0     [As always we had an amazing dinner at Dennys ...
1     [As regular customer of Halais and frequently ...
2     [Giving 3 stars cause there was few waiters, s...
3     [Seems like one of the popular local restauran...
4     [I recently tried the "Shappile Koottu" and wa...
5     [I did enjoy my lunch at Urban Kitchen, this w...
6     [Our favorite Indian restaurant! We've visited...
7     [Amazing Dynamite Burger, it was delicious, ju...
8     [The most tasty and authentic italian buffet\n...
9     [The taste is  good, I tried lahori chargha an...
10    [I can see why this would be a good spot for a...
11    [This is one of two high-end Japanese restaura...
12    ["I was quite surprised when I ordered take aw...
13                                  [i am good service]
14    [I came here with a carnist friend who'd heard...
15    [This place was in my to go for a long time an...
16    [Nice atmosphere, nicely styled with open kitc...
17    [we enjoyed an incredible Asian night at K

In [22]:

from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(df[['rating', 'userRatingCount', 'reviews', 'query']], page_content_column="reviews")
documents = loader.load()
documents

[Document(metadata={'rating': 4.4, 'userRatingCount': 1051, 'query': 'Restaurants in Abu Dhabi'}, page_content="Rating: 4\nI can see why this would be a good spot for a date night, a business lunch, or a dinner with colleagues, thanks to its decent menu and extensive wine list.  During my visit, I ordered escargot, which was very good, pâté, and steak as the main dish.  I was actively searching for a reason to like the restaurant more.  Perhaps I need to visit once more to fully appreciate the restaurant's charm.  However, my initial visit left me feeling somewhat indifferent.\n Rating: 3\nOverall it was a great experience\nWe ordered the set menu option and it’s a great option.\nMany dishes including dessert were offered by the set menu.\nHowever, dessert options were average and the steak was chewy 😞\n\nFor the price I would not rate it above 7.5/10\n Rating: 5\n‼️The atmosphere is amazing ‼️If you’re planning for spending a very classy evening in Abu Dhabi then this restaurant is wh

In [23]:
from langchain_together.embeddings import TogetherEmbeddings

embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import sqlite3
from langchain_chroma import Chroma

fromdisk = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
fromdisk.add_documents(documents=documents)

['bf1cec5b-6566-4c82-bbed-189408202b82',
 'f8e8108b-28cf-4f85-a4ea-a565c7c48983',
 '80fc5a22-c1da-4c6d-a983-1605abdb736b',
 'e228d92f-8602-4ebd-b73a-49f406f8dd6d',
 '150d8ae8-055b-4f4a-b9d6-a7570fdb8a53',
 '22420b9f-a387-49de-96f5-99c1b7dd1283',
 '7a5be2a1-1b94-4eb1-888e-c1fd8cb0576a',
 '7790256e-a6a4-4d3f-b910-b4c3080b2c24',
 'af9d56fb-b89c-448c-81d3-c7453740867d',
 'dc688905-93e3-4ea3-9f1e-8454e3524563',
 '5022f1d1-1b83-486e-a636-ec01558806e1',
 '0e148ae5-d347-4f17-bf24-0af82ef9b24a',
 '85011b5d-3cee-4c86-ac04-b0867ce98bec',
 '43c0cb05-7bb9-4f8b-b91d-b9fa3a0fef82',
 'd95aab6d-28f5-4d44-a738-5c9d665f0c93',
 'ac17f227-ec42-4242-8b18-dc91d70d5112',
 '37f68022-98cf-481b-9711-79a63c2c0c5e',
 '8fd5b91d-9f7a-4bd7-8e51-46de231f9e99',
 '8cfa3570-0564-49fe-a314-e67bc1321b7d',
 'aef0b3d5-5c75-4c63-9b0f-1948d5a7d465']

In [ ]:
# === to load first batch of documents

# # load it into Chroma
# db = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory="./chroma_db")